# Example II.1.4

STYLE ATTRIBUTION
Performing Style analysis of mutual funds by using multiple regression and imposing constraints

In [ ]:
#Import statements
#Python libraries
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import sys

In [ ]:
#FSTK libraries
sys.path.append(os.path.abspath('/Users/yashwantbhambhani/Documents/SHWAYB/FSTK'))
from fstkutil.data_access_util import DataAccess

In [ ]:
da = DataAccess()
print da.getPathOfCSVFile("Historical_Prices_Indices")

In [ ]:
#Arrays for storing input data
arr_historical_prices_mutual_funds_data = []
arr_historical_prices_indices_csv_data = []

#Arrays for storing calculated data
arr_mutual_funds_lognormal_returns = []
arr_indices_lognormal_returns = []

Determine location of script

In [ ]:
script_path = os.path.abspath('__file__') # i.e. /Users/yashwantbhambhani/Documents/SHWAYB/ipython/CarolPythonCode/2.1/2.1.1/__file__
script_dir = os.path.split(script_path)[0] #i.e. /Users/yashwantbhambhani/Documents/SHWAYB/ipython/CarolPythonCode/2.1/2.1.1

Load data from the input file

In [92]:
arr_historical_prices_mutual_funds_csv_data = np.genfromtxt (script_dir + '/data/EX_II.1.4/Historical_Prices_Mutual_Funds_2005_6.csv', delimiter=",")
arr_historical_prices_indices_csv_data = np.genfromtxt (script_dir + '/data/EX_II.1.4/Historical_Prices_Indices_2005_6.csv', delimiter=",")

Yashwant1
[[    nan     nan     nan     nan     nan]
 [    nan   23.63  107.78    8.94   29.33]
 [    nan   23.46  106.9     8.83   29.15]
 ..., 
 [    nan   28.95  131.37   10.44   36.05]
 [    nan   28.9   131.18   10.42   36.  ]
 [    nan   28.78  130.59   10.36   35.84]]
Yashwant2


Calculate Daily Log Normal returns

In [ ]:
#Remove the 'Date' column and first row corresponding to column headers containing stock symbols
#stock_price_data in the line below also includes data for S&P 500 index
arr_historical_prices_mutual_funds_data = arr_historical_prices_mutual_funds_csv_data[1:,1:]
arr_historical_prices_indices_data = arr_historical_prices_indices_csv_data[1:,1:]

for i in xrange(1, len(arr_historical_prices_mutual_funds_data)):
        arr_mutual_funds_lognormal_returns.append(np.log(arr_historical_prices_mutual_funds_data[i] / arr_historical_prices_mutual_funds_data[i-1]))

for i in xrange(1, len(arr_historical_prices_indices_data)):
        arr_indices_lognormal_returns.append(np.log(arr_historical_prices_indices_data[i] / arr_historical_prices_indices_data[i-1]))

#Create dataframe from array of lognormal returns
df_daily_mutual_funds_lognormal_returns = pd.DataFrame(arr_mutual_funds_lognormal_returns)
df_daily_indices_lognormal_returns = pd.DataFrame(arr_indices_lognormal_returns)

#Specify column names to each column of the DataFrame consisting of lognormal returns
df_daily_mutual_funds_lognormal_returns.columns = ['VAA', 'VIT', 'FAA', 'FID']
df_daily_indices_lognormal_returns.columns = ['R1000V', 'R1000G', 'R2000V', 'R2000G']

Calculate the difference in lognormal returns and risk factor coefficient adjusted index returns

In [ ]:
#Create an all-compassing universe of regression coefficients for all the four style factors
arr_reg_coeff_for_style_factors = []

for reg_coeff_for_style_factor_r1000v in np.arange(0, 1.001, 0.02):
    for reg_coeff_for_style_factor_r1000g in np.arange(0, 1.001, 0.02):
        for reg_coeff_for_style_factor_r2000v in np.arange(0, 1.001, 0.02):
            for reg_coeff_for_style_factor_r2000g in np.arange(0, 1.001, 0.02):
                if (reg_coeff_for_style_factor_r1000v + reg_coeff_for_style_factor_r1000g + reg_coeff_for_style_factor_r2000v + reg_coeff_for_style_factor_r2000g == 1):
                    arr_reg_coeff_for_style_factors.append([reg_coeff_for_style_factor_r1000v, reg_coeff_for_style_factor_r1000g, reg_coeff_for_style_factor_r2000v, reg_coeff_for_style_factor_r2000g])

In [ ]:
reg_coeff_adjusted_lognormal_returns = np.array(arr_reg_coeff_for_style_factors).dot(np.array(arr_indices_lognormal_returns).transpose())

arrT = np.array(arr_mutual_funds_lognormal_returns).transpose()

arrT_first = arrT[0]
arrT_first_diff = arrT_first - reg_coeff_adjusted_lognormal_returns
arrT_first_diff_square = np.square(arrT_first_diff)

arrT_second = arrT[1]
arrT_second_diff = arrT_second - reg_coeff_adjusted_lognormal_returns
arrT_second_diff_square = np.square(arrT_second_diff)

arrT_third = arrT[2]
arrT_third_diff = arrT_third - reg_coeff_adjusted_lognormal_returns
arrT_third_diff_square = np.square(arrT_third_diff)

arrT_fourth = arrT[3]
arrT_fourth_diff = arrT_fourth - reg_coeff_adjusted_lognormal_returns
arrT_fourth_diff_square = np.square(arrT_fourth_diff)

x1_min_location = np.where(np.sum(arrT_first_diff_square, axis = 1) == np.min(np.sum(arrT_first_diff_square, axis = 1)))[0]
x2_min_location = np.where(np.sum(arrT_second_diff_square, axis = 1) == np.min(np.sum(arrT_second_diff_square, axis = 1)))[0]
x3_min_location = np.where(np.sum(arrT_third_diff_square, axis = 1) == np.min(np.sum(arrT_third_diff_square, axis = 1)))[0]
x4_min_location = np.where(np.sum(arrT_fourth_diff_square, axis = 1) == np.min(np.sum(arrT_fourth_diff_square, axis = 1)))[0]

print "2005_6_VAA", arr_reg_coeff_for_style_factors[x1_min_location]
print "2005_6_VIT", arr_reg_coeff_for_style_factors[x2_min_location]
print "2005_6_FAA", arr_reg_coeff_for_style_factors[x3_min_location]
print "2005_6_FID", arr_reg_coeff_for_style_factors[x4_min_location]